In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers 

runcycleid = etl_helpers.start_run_cycle("factRequestInvoices")

try:
    today = str(datetime.date.today())

    df_existing = spark.sql("SELECT max(TIMESTAMP(runcycleendat)) as runcycleendat from dimruncycle where packagename = 'factRequestInvoices' and success = 't'")
    df_existing.show()
    maxcreatedate = df_existing.first().runcycleendat
    print(maxcreatedate)
    maxcreatedate_str = maxcreatedate.strftime("%Y-%m-%d %H:%M:%S")


    query = f"""
        select * from
        (SELECT *
                FROM (
                    SELECT 
                    get_json_object(feedata, '$.estimatedtotaldue') as invoiceamount, 
                    get_json_object(feedata, '$.amountpaid') as chargedamount, 
                    get_json_object(feedata, '$.feewaiveramount') as feewaived, 
                    created_at,
                    createdby,
                    cfrfeeid,
                    ministryrequestid,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY cf.ministryrequestid 
                            ORDER BY cf.created_at DESC
                        ) AS rn
                    FROM foi_mod.foirequestcfrfees cf
                    where created_at > '{maxcreatedate_str}'
                ) sub
                WHERE rn = 1) sq

        join 

        (SELECT *
                FROM (
                    SELECT 
                    created_at,
                    createdby,
                    ministryrequestid,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY cf.ministryrequestid 
                            ORDER BY cf.created_at DESC
                        ) AS rn
                    FROM foi_mod.foirequestcfrfees cf
                    where cfrfeestatusid = 2 --approved
                ) sub
                WHERE rn = 1) sq2 on sq.ministryrequestid = sq2.ministryrequestid

        join

        (SELECT *
                FROM (
                    SELECT 
                    created_at,
                    createdby,
                    ministryrequestid,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY cf.ministryrequestid 
                            ORDER BY cf.created_at asc
                        ) AS rn
                    FROM foi_mod.foirequestcfrfees cf
                ) sub
                WHERE rn = 1) sq3 on sq3.ministryrequestid = sq2.ministryrequestid

        join 
        (select foirequest_id, foiministryrequestid from foi_mod.foiministryrequests) sq4 on sq4.foiministryrequestid = sq.ministryrequestid
        """

    print(query)

    df = spark.sql(query)
    print(df.count())
    df.show()


    # order of columns here is important!
    df_mapped = df.selectExpr(
        "foirequest_id as foirequestid",
        f"{runcycleid} as runcycleid",
        "cfrfeeid as invoiceid",
        "NULL as invoicenumber",
        "sq3.createdby AS createdby",
        "sq3.created_at AS createddate",
        "sq.createdby AS modifiedby",
        "sq.created_at AS modifieddate",
        "'' as comments",
        "'' AS correspondenceid",
        "invoiceamount AS invoiceamount",
        "chargedamount AS chargedamount",
        "'' as incurredamount",
        "feewaived AS feewaived",
        "'' as admincostpercentage",
        "'' as chargedadmincostamount",
        "NULL as incurredadmincostamount",
        "NULL as feewaivedadmincost",
        "'' as annotationtext",
        "sq2.created_at AS invoiceddate",
        "'' as approvalstatus",
        "sq2.createdby AS approvedby",
        "sq2.created_at AS approveddate",
        "'' as activeflag",
    )
    df_mapped.show()
    df_mapped.write.format("delta").mode("append").option("mergeSchema", "false").insertInto("factrequestinvoices")  
    etl_helpers.end_run_cycle(runcycleid, 't', "factRequestInvoices")
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', "factRequestInvoices", "Credentials not available")
except Exception as e:    
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', "factRequestInvoices")
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', "factRequestInvoices", f"An error occurred: {e}")